In [1]:
import json
data_dir = "C:/Users/Kshitij Alwadhi/Documents/GitHub/Clarinet/Data/Json/2018_clipped/"+'notes.json'
queries = "C:/Users/Kshitij Alwadhi/Documents/GitHub/Clarinet/Data/Json/2018_queries/"+'notes.json'


In [2]:
def splitNotes(notes):
    new_notes = []
    for note in notes:
        start = note[1]/1000.
        end = note[2]/1000.
        startSec = int(start)
        endSec = int(end)

        if startSec == endSec:
            new_notes.append(note)
        else:
            new_notes.append([note[0], note[1], (startSec+1)*1000, note[3]])
            for i in range(startSec+1, endSec-1):
                new_notes.append([note[0], int(i*1000), int(i+1)*1000, note[3]])
            new_notes.append([note[0], int(endSec*1000), note[2], note[3]])
    return new_notes


def getStrides(notes, size):
    notes = sorted(notes, key=lambda x: x[1])
    chunks = []
    for _ in range(len(notes)):
        chunks.append([])

    curT = 0
    nextidx = 0
    flag = False

    i = 0
    while(i < len(notes)):
        st = notes[i][1]
        maxT = curT + size
        if st >= maxT:
            curT += 1000
            i = nextidx
            flag = False
            continue

        if st % 1000 == 0 and flag == False and st != curT:
            nextidx = i
            flag = True

        pos = int(curT/1000)
        chunks[pos].append(notes[i])
        i += 1
    ans = []
    for c in chunks:
        if len(c) > 0:
            ans.append(c)
    return ans


In [3]:
with open(data_dir, 'r') as f:
    fname_to_notes = json.load(f)


In [4]:
midiEt_to_note = {
    12: "C",
    13: "C#",
    14: "D",
    15: "D#",
    16: "E",
    17: "F",
    18: "F#",
    19: "G",
    20: "G#",
    21: "A",
    22: "A#",
    23: "B"
}


# a function which converts a sequence of midiEt to a sequence of notes
def midiEt_to_note_sequence(midiEt_sequence):
    note_sequence = []
    for midiEt in midiEt_sequence:
        num = midiEt % 12
        note_sequence.append(midiEt_to_note[num + 12])
    return "".join(note_sequence)


In [ ]:
from .mongeau_sankoff import similarity,Note

queries=[Note("G",2,rest=False)]

# t=total_time/(4*tempo)
# duration=int(end-start/t)
